# $k$-NN Module
In this module, we introduce some fundamental concepts in machine learning (ML) while also introducing the $k$-NN model: one of the most basic non-parametric supervised learning models.

The main idea of the $k$-NN model is based on an idea popularized by Waldo Tobler: "everything is related to everything else, but near things are more related than distant things." Essentially, things that are closer to each other are more similar to each other. Using this idea gives us a method for prediction given that we have observed some data. Now let's formalize this idea further in this module.

The $k$-NN algorithm uses those observations in the training set $\mathcal{T}$ closest in feature space to a query point $x_0$ to form the prediction $\hat{y}$. Specifically, the $k$-nearest neighbor classifier for $\hat{y}$ is defined as:
$$
\hat{y}(x_0) = \argmax_{c \in \mathcal{C}} \sum_{x_i \in \mathcal{N}_k(x_0)} \delta_{y_i, c}
$$
where $\mathcal{C}$ is the set of class labels, $\mathcal{N}_k(x_0)$ is the neighborhood of $x_0$ defined by the $k$ closest points in $\mathcal{T}$, and $\delta_{y_i, c}$ is the *Kronecker delta*:
$$
\delta_{y_i, c} = 
\begin{cases} 
1 & \text{if } y_i = c \\ 
0 & \text{otherwise} 
\end{cases}
$$

Closeness implies some notion of distance and a corresponding metric, which in our case we assume is the *$\ell^2$ norm* (Euclidean distance). In words, given a query point $x_0$, we find the $k$ training observations closest to $x_0$ in feature space, and classify using majority vote among the $k$ neighbors. In the case of ties, they are broken at random.

Now that we've introduced some of the theory for $k$-NN, let's experiment with the model through a few exercises. The first section of this module is a dedicated tutorial to [`scikit-learn`](https://scikit-learn.org/stable/index.html) that also teaches some basic ML concepts using $k$-NN as the motivating example.

We first import the necessary packages for this module.

In [4]:
import polars as pl
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import seaborn.objects as so
import seaborn as sns
import matplotlib.pyplot as plt
from utils import view_boundary, view_model_size
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

sns.set_theme(style="whitegrid", palette="tab10")

In order to talk about modeling, we first need to talk about data. The data we will be using for the tutorial portion is the [*Iris*](https://archive.ics.uci.edu/dataset/53/iris) dataset, which can be found in the `iris.csv` file in the `data` directory. The data consists of $150$ observations and $5$ variables:

| Column Name | Description |
|------------|-------------|
| `sepal_length` | Sepal length (centimeters) |
| `sepal_width` | Sepal width (centimeters) |
| `petal_length` | Petal length (centimeters) |
| `petal_width` | Petal width (centimeters) |
| `species` | Iris species (setosa, versicolor, virginica) |

Our goal with this dataset will be to classify three different plant species based on four features: the flower's sepal length, sepal width, petal length, and petal width. As such, our target (class) variable is `species`.

With these logistical details out of the way, we now read in the dataset using [`read_csv`](https://docs.pola.rs/api/python/dev/reference/api/polars.read_csv.html) from the [`Polars`](https://pola.rs/) package.

In [ ]:
col_names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]
iris = pl.read_csv("../data/iris.csv", has_header=False, new_columns=col_names)[:-1]

Before we begin any modeling, it's always a good idea to do some exploratory data analysis (EDA) and see what kind of data we are dealing with. Looking at the data beforehand can provide us with insights into how we should model our data, or make us aware of any patterns in the data. 

One of the first things we may want to look at is the proportions of our label class (in our case the species of the flowers). Doing so will help us determine if there are any class imbalances, in which case, we will need to employ addition techniques to elevate this issue.

---
#### **Prelude Question**
Why might class imbalances negatively impact the modeling process?

**Answer:**

---

In [ ]:
so.Plot(iris.to_pandas(), x="species", color="species").add(so.Bar(), so.Count()).show()
print(iris.group_by("species").len())

From the bar plot above, we note that there appears to be an equal number of observations for each species. Therefore, class imbalance should not be an issue with this dataset. Some other useful things to examine are the distributions of the features, and the relationships between each of them. Fortunately, a pairs plot allows us to do both of these things simultaneously. As such, we provide one below.

In [ ]:
sns.pairplot(iris.to_pandas(), hue="species")
plt.show()

By construction, a pairs plot is symmetric, so we only need to pay attention to the plots below the main diagonal as the ones above the diagonal are redundant. Looking at the plot, we see that for almost every pair of features plotted, the observations are clustered together by species, making $k$-NN an ideal model to fit to the data. This is to be expected since the very purpose of a species classification system is to distinguish between different species based on their physical characteristics.

## $k$-NN Modeling
After doing some EDA, we now need to pre-process the data before we can begin modeling. 

### Normalizing Features
Our first order of business is to normalize our features. Since the performance of $k$-NN is entirely dependent on a distance metric, it's important that our notion of distance is standardized across all our features. Otherwise, our features will be on different scales, which would induce a magnitude bias in our model. In other words, features that on average have higher values would dominate the model, marginalizing the effect of other features that have on average smaller values. Our strategy for standardizing will be to map values to a $[0, 1]$ range. This is also called [*min-max scaling*](https://en.wikipedia.org/wiki/Feature_scaling#Rescaling_(min-max_normalization)) and can be done by applying the following formula to each column in our dataset (excluding the class column):
$$
X_{ij,\text{normalized}} = \frac{X_{ij} - X_{j,\text{min}}}{X_{j,\text{max}} - X_{j,\text{min}}}
$$
where $X_{ij}$ is the value of the $i \text{th}$ observation and $j\text{th}$ feature. The programming for this is a bit involved, so fortunately [`scikit-learn`](https://scikit-learn.org/stable/index.html) provides us with the [`MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) class. We simply need to create an instance of [`MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) and call the [`fit_transform`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler.fit_transform) method on it, passing our features `X` as a parameter.

### Encoding Class Labels
Next, we need to address the fact that machine learning models don't handle non-numeric data very well. Since our class labels are strings, we need to convert them to integer values. We'll use the following mapping:
\begin{align*}
&\text{Iris-setosa} \to 0 \\
&\text{Iris-versicolor} \to 1 \\
&\text{Iris-virginica} \to 2
\end{align*}
Again, we don't need to do any of this manually. The [`scikit-learn`](https://scikit-learn.org/stable/index.html) package provides the [`LabelEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) class that handles this for us. All we need to do is call the [`fit_transform`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder.fit_transform) method on a [`LabelEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) instance and pass our labels `y` as a parameter into the method.

### Train-Test Split
Finally, we need to split the data into a ***training set*** and a ***test set*** (this is also known as the *holdout method*). As the names imply, the training set is the data we use to train our model, and the test set is the data we use to test our model. To create these two sets, we first shuffle the dataset to make sure that the order in which the observations appear do not affect the learning process. Next, we randomly partition our dataset into two disjoint subsets. It is essential that these two subsets are disjoint to prevent data leakage. Here, we will use a $70/30$ split (i.e., $70\%$ of the observations will be in the training set and $30\%$ in the test set). 

If this sounds like a lot, that's because it is! Fortunately, [`scikit-learn`](https://scikit-learn.org/stable/index.html) provides the [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function that handles all the troublesome logic for us. We simply pass in our features `X` and labels `y`, and specify the split size using either the `train_size` or `test_size` parameter. Additionally, we set the `random_state` parameter to `42` to ensure reproducibility. This guarantees that we get the same splits every time we run the code.

All of these pre-processing steps are shown in the code below.

In [ ]:
# Splitting data into features and class labels
X, y = iris[:, :-1].to_numpy(), iris[:, -1].to_numpy()

# Normalizing data onto [0, 1] range
X_scaled = MinMaxScaler().fit_transform(X)

# Encoding class labels as integers
y_encoded = LabelEncoder().fit_transform(y)

# Partitioning data into training and test set
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.3, random_state=42
)

Now that our data is all ready to go, we can get to the actual modeling portion. But first, we need to briefly explain how [`scikit-learn`](https://scikit-learn.org/stable/index.html) actually works. 

The general process for developing supervised learning models is:
1. Train the model on the training data
2. Evaluate the model's performance on the test data
3. If the model generalizes well on the test data, use the trained model for inference on new, unseen data

In [`scikit-learn`](https://scikit-learn.org/stable/index.html), all ML models are called [*estimators*](https://scikit-learn.org/stable/glossary.html#term-estimator) (note: this is not to be confused with the notion of estimators from statistics). These estimators are implemented as classes that inherit from a *base estimator* class and use what is called a [*mixin*](https://en.wikipedia.org/wiki/Mixin) class to add specific functionality. For our purposes, since we are working in the classification setting, the models we will be using are subclasses of both the [`BaseEstimator`](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html) class and the [`ClassifierMixin`](https://scikit-learn.org/stable/modules/generated/sklearn.base.ClassifierMixin.html) class. The [`BaseEstimator`](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html) class provides core functionality like parameter management, while the [`ClassifierMixin`](https://scikit-learn.org/stable/modules/generated/sklearn.base.ClassifierMixin.html) adds classifier-specific methods such as [`score`](https://scikit-learn.org/stable/glossary.html#term-score), which computes classification accuracy.

The typical workflow for using these estimators follows these steps:

1. **Initialize the model**: Create an instance of the estimator class with the desired hyperparameters
2. **Fit the model**: Train the model on the training data using the [`fit`](https://scikit-learn.org/stable/glossary.html#term-fit) method
3. **Evaluate the model**: Assess the model's performance on the test data using the [`score`](https://scikit-learn.org/stable/glossary.html#term-score) method or other metrics (e.g. accuracy, precision, $R^2$, or $\text{MSE}$)
4. **Make predictions**: Use the trained model to predict labels on new data using the [`predict`](https://scikit-learn.org/stable/glossary.html#term-predict) method

Therefore, all [*classifiers*](https://scikit-learn.org/stable/glossary.html#term-classifier) that inherit from [`BaseEstimator`](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html) and [`ClassifierMixin`](https://scikit-learn.org/stable/modules/generated/sklearn.base.ClassifierMixin.html) must implement the [`fit`](https://scikit-learn.org/stable/glossary.html#term-fit), [`score`](https://scikit-learn.org/stable/glossary.html#term-score), and [`predict`](https://scikit-learn.org/stable/glossary.html#term-predict) methods. This architecture helps ensure consistency across all classification models, as any classifier will have the same basic interface, making it easy to swap models or compare different approaches.

With this framework in mind, let's build our $k$-NN model. Here, we chose the default value $k = 5$.

In [ ]:
# Initializing model with hyperparameters (default: k = 5)
model = KNeighborsClassifier()

# Fitting model to training set
model.fit(X_train, y_train)

# Evaluating model using test set and accuracy metric
model.score(X_test, y_test)

Wow, our model achieved perfect accuracy! It's almost as if this was intentional...

---
#### **Postlude Question**
Can you think of some reasons why the model might have performed so well on the data?

**Answer:**

---

## Computational Considerations
A few words on the $k$-NN algorithm. You may have noticed from our formulation of the $k$-NN algorithm at the beginning of the module that there isn't really any training going on in the algorithm. The model simply takes in a new data instance and then performs a calculation.

This is because $k$-NN is what's called a *lazy*, *memory-based* (or *instance-based*) learning algorithm. Let's break this definition down: 
- When we say that $k$-NN is a ***lazy*** learner, we mean that computation is deferred until a query is made to the model. In other words, there's no upfront "learning phase". All of the work is done at prediction time. 
- When we say that $k$-NN is a ***memory-based*** (***instance-based***) learning algorithm, we mean that there is essentially no model to fit. The algorithm's predictions are simply based on comparisons of a query point with data points in the training set, rather than on a global model.

To make this more concrete, let's do an experiment to estimate the space complexity of $k$-NN. We perform two simulations: one where we fix the number of features $d$ and vary the number of samples $n$, and another where we do the reverse. Run the code below and answer the following question.

In [ ]:
models = {
    r"$k$-NN": KNeighborsClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=200),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=10),
}

n_pow = 5
param_values = np.logspace(1, n_pow, n_pow, base=10, dtype=int)

df_samples, df_features = view_model_size(models, param_values, random_state=67)

print(df_samples)
print(df_features)

---
#### **Postlude Question**
From the plots, what is the *space* complexity of the $k$-NN algorithm? Based on our discussion above, why does the space complexity grow in this manner?

**Answer:**

---

# Bias-Variance Tradeoff and Decision Boundaries
We now focus on two separate but deeply intertwined concepts in ML: the bias-variance tradeoff and decision boundaries. These concepts are connected through how we evaluate model performance. The bias-variance tradeoff describes *why* models make errors, while decision boundaries provide a visual way to *observe* these errors.

## Bias-Variance Tradeoff
Recall from statistics that the mean squared error $\text{(MSE)}$ for an estimator $\hat{f}$ at a query point $x_0$ is defined as

$$
\text{MSE}(x_0) = \mathbb{E}_\mathcal{T}\left[\left(f(x_0) - \hat{f}(x_0)\right)^2\right]
$$

where $f(x_0)$ is the true value, $\hat{f}(x_0)$ is the predicted value, and the expectation is taken over all possible training sets $\mathcal{T}$. Though this formulation is typically used in the regression setting, its implications also apply to classification.

With some algebra, the $\text{MSE}$ can be decomposed into three components:

$$
\text{MSE}(x_0) = \text{Bias}^2\left(\hat{f}(x_0)\right) + \text{Var}\left(\hat{f}(x_0)\right) + \sigma^2_\epsilon
$$

where:
- **Bias**: measures how far off the average prediction is from the true value.
- **Variance**: measures how much the predictions vary for different training sets.
- **Irreducible error** ($\sigma^2_\epsilon$): represents noise in the data that cannot be reduced by any model.

The key insight is that there's a tradeoff: as we reduce the bias of our model, we typically increase its variance, and vice versa. Our goal is to find the model complexity that minimizes the total error (i.e., the $\text{MSE}$). We should note that one way to think about ML is that we are trying to estimate some function $f$ that maps from our data $X$ to our class labels $Y$, where $f$ is some fixed, unknown function. As such, the goal of ML is to estimate this function using an estimator $\hat{f}$. Since we are dealing with an estimator, we have all of the tools of statistics at our disposal.

## Decision Boundaries
To understand how this tradeoff appears in practice, we turn to the concept of decision boundaries. Another way to think about ML is that we are trying to learn the shape of a *decision boundary*. A ***decision boundary*** is a hypersurface that partitions the feature space into regions, with each region corresponding to a particular class. Points on one side of the boundary are classified as one class, while points on the other side are classified as another. Here, a ***hypersurface*** is defined to be a generalization of the concepts of a hyperplane, plane curve, and surface. For points exactly on the decision boundary, the class label is ambiguous; the model is equally uncertain about which class to assign. In the simplest case with two features and binary classification, the decision boundary is a curve (or a line if the classification problem is *linear*). More generally, in $d$ dimensions, the decision boundary is a $(d-1)$-dimensional hypersurface. We consider a classification problem to be ***linear*** if and only if the decision boundary is a hyperplane, which also means the classes are ***linearly separable***. However, most real-world problems have non-linear decision boundaries.

Decision boundaries provide a visual tool for understanding the bias-variance tradeoff. To investigate this phenomenon for the $k$-NN algorithm, we'll use decision boundaries as a visual diagnostic. Below, we create a synthetic binary classification problem and generate decision boundaries for various values of $k$. Run the code chunk below and answer the questions that follow.

**Note on the plots**: The color intensity in each region indicates the model's confidence; lighter colors represent greater uncertainty in the predictions, while darker colors indicate higher confidence.

In [ ]:
X, y = make_classification(n_features=2, n_redundant=0, flip_y=0.25, random_state=67)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

k_values = [1, 3, 7, 11, 21, 31, 51, 71, X_train.shape[0] - 1, X_train.shape[0]]
n_cols = 5
n_plots = len(k_values)
n_rows = np.ceil(n_plots / n_cols).astype(int)

fig, axs = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
axs = np.ravel(axs)

for i, k in enumerate(k_values):
    view_boundary(
        KNeighborsClassifier(k).fit(X_train, y_train),
        X_train,
        X_test,
        y_train,
        y_test,
        f"$k$-NN Decision boundary for $k=${k}",
        axs[i],
    )

for i in range(n_plots, n_rows * n_cols):
    axs[i].axis("off")

plt.tight_layout()
plt.show()

---
#### **Question 1**
Analyze how the decision boundary changes as $k$ varies:
- How does varying $k$ affect the model's bias and variance?
- The ***complexity*** of a model can be approximated by analyzing its decision boundary. A more complex decision boundary is indicative of a more complex model. For which values of $k$ does the decision boundary become more complex?
- At $k = 1$, the decision boundary contains small "islands" of one class surrounded by another. Why does this occur, and why is it problematic for generalization?
- Compare the decision boundaries at $k = 3$ and $k = 21$. Which value would be preferable if the true underlying decision boundary were linear? What if it were highly non-linear?

**Answer:**

---

#### **Question 2**
Consider two extreme cases of $k$:
- What would happen if we used only the training set to evaluate the model when $k = 1$?
- What happens to the model's predictions when $k = |\mathcal{T}|$ (i.e., when $k$ equals the size of the training set)?

**Answer:**

---

#### **Question 3**
We say a model is ***underfitting*** when it performs poorly on both the training and test data and ***overfitting*** when it performs well on the training data but poorly on the test data. Based on your analysis, what value (or range of values) of $k$ would achieve the highest test set accuracy? Explain your answer in terms of bias and variance.

**Answer:**

---

#### **Question 4**
If we were to add new test points in regions where the decision boundary changes significantly between different $k$ values, what would this tell us about prediction uncertainty?

**Answer:**

---

#### **Question 5**
We're currently using the $\ell^2$ norm (Euclidean distance). How might the decision boundaries change if we:
- Used the *$\ell^1$ norm* (Manhattan distance) instead?
- Weighted $x_1$ and $x_2$ differently?

**Answer:**

---

# Palmer Penguins
Now let's apply our theory and knowledge of $k$-NN to an another dataset. Here, we'll be using the [*Palmer Penguins*](https://allisonhorst.github.io/palmerpenguins/) data, which can be found in the `penguins.csv` file in the `data` directory. The data consists of $334$ observations and $8$ variables:

| Column Name | Description |
|------------|-------------|
| `species` | Penguin species (Adélie, Chinstrap, and Gentoo) |
| `island` | Island in Palmer Archipelago, Antarctica (Biscoe, Dream, or Torgersen) |
| `bill_length_mm` | Bill length (millimeters) |
| `bill_depth_mm` | Bill depth (millimeters) |
| `flipper_length_mm` | Flipper length (millimeters) |
| `body_mass_g` | Body mass (grams) |
| `sex` | Penguin sex (female, male) |
| `year` | Study year (2007, 2008, or 2009) |

Similar to the [*Iris*](https://archive.ics.uci.edu/dataset/53/iris) dataset, our goal is to predict a penguin's species from its physical characteristics. As such, the columns of interest are `bill_length_mm`, `bill_depth_mm`, `flipper_length_mm`, and `body_mass_g`, and our target variable is `species`.

---
#### **Prelude Question**
As with the [*Iris*](https://archive.ics.uci.edu/dataset/53/iris) dataset, perform some EDA on the [*Palmer Penguins*](https://allisonhorst.github.io/palmerpenguins/) data before doing any modeling to gain insights into any underlying structures/patterns in the data. Note: the EDA presented in the tutorial is the bare minimum; you should do more exploration in your own analysis.

**Answer:**

---

Now that you've explored the data a bit, let's perform an experiment. Our goal is to determine the impact of the parameter $k$ on the algorithm's performance when used to classify observations in the training and test set.

You will construct two plots showing the accuracy of the $k$-NN model as a function of $k$: one evaluated on the *training set* and one on the *test set*. For both plots, vary $k$ from $1$ to $101$ using only odd numbers $(1, 3, \ldots, 101)$. This process is known as *hyperparameter tuning* (granted, a pretty rudimentary way of doing it). To account for variability, train a $k$-NN model $20$ times for each value of $k$. This will produce $20$ accuracy estimates on the training data and $20$ accuracy estimates on the test data for each $k$ value.

Once you've done this, answer the following questions:

---
#### **Question 1**
Create two plots showing model performance as a function of $k$:
- **Plot 1**: Average accuracy on the *training set* vs. $k$
- **Plot 2**: Average accuracy on the *test set* vs. $k$

For both plots, display $k$ on the horizontal axis and average accuracy on the vertical axis. Add error bars to each point to show the standard deviation across the $20$ trials.

**Answer:**

---
#### **Question 2**
Analyze the shape of each plot in terms of bias and variance:
- Why does the training set accuracy curve look the way it does as $k$ increases?
- Why does the test set accuracy curve look the way it does as $k$ increases?
- Based on your plots, identify the ranges of $k$ where the model is: (a) underfitting, (b) overfitting, and (c) achieving a good balance. For each case, explain your answer in terms of bias and variance.

**Answer:**

---
#### **Question 3**
Based on your analysis of the training and test accuracy curves, what value (or range of values) of $k$ would you recommend using for this classification task? Justify your choice by considering:
- The tradeoff between training and test performance
- The stability of performance (as indicated by error bars)

**Answer:**

---
#### **Question 4**
Choose *two* features from the dataset and visualize how the decision boundary changes as you vary $k$ using the provided `view_boundary` function. Based on your visualizations, identify which value of $k$ produces the best decision boundary for your chosen features. Explain your rationale for selecting these features, justify your choice of $k$, and discuss how effectively the model separates the class labels in this two-dimensional space. This process of choosing informative features is known as *feature selection*. 

Hint: Some feature pairs will yield clearer decision boundaries than others.

**Answer:**

---
#### **Question 5**
In the experiments above, you normalized the features before running $k$-NN to ensure all features are considered equally important when computing distances. Now, investigate the impact of omitting feature normalization:
- Repeat Question $1$ by creating a plot showing average accuracy (with standard deviation error bars) on the *test set* as a function of $k$, but this time run $k$-NN directly on the original unnormalized data.
- Identify the best value of $k$ for the unnormalized version based on test set performance.
- Compare the performance of $k$-NN with and without feature normalization. Which performs better, and why do you think this is the case? Consider which features might dominate the distance calculations when normalization is omitted.

**Answer:**

---

# Curse of Dimensionality
Since the $k$-NN algorithm is entirely dependent on distance, any issues that arise when working in higher-dimensional spaces will directly affect the model's performance.

For this simulation, we'll demonstrate the ***curse of dimensionality***. In the context of ML, this phenomenon occurs when we have a fixed number of training examples but an increasing number of dimensions in the feature space. As the number of dimensions increase, data points become increasingly sparse, making it difficult for distance-based methods like $k$-NN to find meaningful nearest neighbors.

To illustrate this, we'll examine the relationship between a unit *hypercube* and its inscribed unit *hypersphere* as we increase the number of dimensions. Specifically, we'll estimate what proportion of the hypercube's volume is occupied by the inscribed hypersphere.

But first, a few definitions: 
- We define a ***hypercube*** to be a $p$-dimensional analogue of a cube. When $p = 1$, a hypercube is simply a line segment; when $p = 2$, it's a square; and when $p = 100$, it's a $100$-dimensional cube. 
- Similarly, we define a ***hypersphere*** to be a $p$-dimensional analogue of a sphere. When $p = 1$, a hypersphere is a line segment; when $p = 2$, it's a circle; and in higher dimensions, it generalizes accordingly.

With that out of the way, let's get to the actual experiment! Write a simulation that implements the following:

```
FOR p = 1 to p_MAX:
    // Generate random points uniformly in unit hypercube
    // Calculate distance from origin for each point
    // Determine which points fall inside unit hypersphere
    // Calculate proportion of points inside hypersphere
END FOR
```

Be sure to plot the results of your simulation and to set the `seed` parameter for `default_rng` to be `67` to ensure reproducibility. Once you've finished your implementation, answer the following questions:

---
#### **Question 1**
Describe the pattern you observe as the number of dimensions increases. What proportion do you observe in dimensions $1$ and $10$? Is the change linear, exponential, or something else?

**Answer:**

---
#### **Question 2**
From the simulation, we see that as the number of dimensions increase, the hypersphere occupies a decreasing proportion of the hypercube's volume. Where is the remaining volume located, and what does this suggest about where randomly sampled data points are likely to be found in high-dimensional space?

**Answer:**

---
#### **Question 3**
In high-dimensional spaces, the distance to the nearest neighbor approaches the distance to the farthest neighbor. Using your understanding of how volume distributes in hypercubes, explain:
- Why this counterintuitive result occurs
- What it means for the concept of "nearest" neighbors
- How this phenomenon explains why $k$-NN and other distance-based algorithms struggle in high-dimensional spaces

**Answer:**

---
#### **Question 4**
In a $2$-dimensional space, suppose you have a dataset with $100$ training examples uniformly distributed. To maintain the same data density in a $10$-dimensional space, roughly how many training examples would you need? Show your reasoning. What does this imply for real-world high-dimensional datasets?

**Answer:**

---
#### **Question 5**
Many real-world datasets (like images or text) are naturally high-dimensional. Given what you've learned about the curse of dimensionality, what strategies might help mitigate this issue when using $k$-NN? Consider:
- Data preprocessing techniques (e.g., PCA, feature selection)
- Algorithm modifications
- How these approaches relate to the volume distribution you observed in the simulation

**Answer:**

---

# *Fin*